# A2 - Bias in Data Assignment

In [1]:
# import libraries 

import pandas as pd
import numpy as np

In [2]:
# import two datasets

page_data = pd.read_csv("page_data.csv")

WPDS_data = pd.read_csv("WPDS_2020_data.csv")

In [3]:
# Check the data has been imported properly

page_data.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [4]:
# Check the WPDS data has been imported properly 

WPDS_data.head()

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000


## Data Cleaning

In [5]:
# Remove rows where "page" starts with "Template" in page_data 

page_data = page_data[~page_data.page.str.contains("Template:")]

page_data

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


In [6]:
# remove rows where "Name" contains all caps values  in WPDS_data

WPDS_clean = WPDS_data[~WPDS_data['Name'].str[:].str.isupper()]

# Note there is still one row left where the Type is specified as sub-region, need to remove this row as well
WPDS_clean = WPDS_clean[WPDS_clean['Type']=='Country']

WPDS_clean

,FIPS,Name,Type,TimeFrame,Data (M),Population
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
5,LY,Libya,Country,2019,6.891,6891000
6,MA,Morocco,Country,2019,35.952,35952000
7,SD,Sudan,Country,2019,43.849,43849000
...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000
230,SB,Solomon Islands,Country,2019,0.715,715000
231,TO,Tonga,Country,2019,0.099,99000
232,TV,Tuvalu,Country,2019,0.010,10000


## Getting Article Quality Predictions 

In [7]:
# import the libaries

import json
import requests
import os

In [8]:
# Set the endpoint for using the API 
endpoint = 'https://ores.wikimedia.org/v3/scores/enwiki/?models=articlequality&revids={rev_id}'

In [9]:

# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/Sabrinawang06',
    'From': 'lxw5332@uw.edu'
}


# Define the API call
def api_call(endpoint, rev_id):
    call = requests.get(endpoint.format(rev_id = rev_id), headers=headers)
    response = call.json()
    return response




In [10]:
# create batch of 50 rev_id and loop over the whole table of page_data

res = [] # valid API returns 
error_log = [] # API returns with error 
n= page_data.shape[0]

for i in range(0,n,50):
    if i+50 > n:
        batch = api_call(endpoint, "|".join(str(x) for x in page_data.rev_id.iloc[i:n]))
    else:
        batch = api_call(endpoint, "|".join(str(x) for x in page_data.rev_id.iloc[i:i+50]))
        
    
    
    for j in batch['enwiki']['scores'].keys():
        try:     
            res.append([j, batch['enwiki']['scores'][j]['articlequality']['score']['prediction']])
        except KeyError:
            error_log.append([j,batch['enwiki']['scores'][j]])
            pass
        

            

In [11]:
# Conver previous output into dataframe and rename the columns 
prediction = pd.DataFrame(res).rename(columns={0: "rev_id", 1: "prediction"})
prediction

,rev_id,prediction
0,355319463,Stub
1,393276188,Stub
2,393822005,Stub
3,395521877,Stub
4,395526568,Stub
...,...,...
46420,807481636,C
46421,807482007,GA
46422,807483006,C
46423,807483153,GA


In [12]:
# Check the error log 
error_log

[['516633096',
  {'articlequality': {'error': {'message': 'RevisionNotFound: Could not find revision ({revision}:516633096)',
     'type': 'RevisionNotFound'}}}],
 ['550682925',
  {'articlequality': {'error': {'message': 'RevisionNotFound: Could not find revision ({revision}:550682925)',
     'type': 'RevisionNotFound'}}}],
 ['627547024',
  {'articlequality': {'error': {'message': 'RevisionNotFound: Could not find revision ({revision}:627547024)',
     'type': 'RevisionNotFound'}}}],
 ['636911471',
  {'articlequality': {'error': {'message': 'RevisionNotFound: Could not find revision ({revision}:636911471)',
     'type': 'RevisionNotFound'}}}],
 ['669987106',
  {'articlequality': {'error': {'message': 'RevisionNotFound: Could not find revision ({revision}:669987106)',
     'type': 'RevisionNotFound'}}}],
 ['671484594',
  {'articlequality': {'error': {'message': 'RevisionNotFound: Could not find revision ({revision}:671484594)',
     'type': 'RevisionNotFound'}}}],
 ['680981536',
  {'art

## Combining the Datasets

In [79]:
# Conver the rev_id type in the prediction dataframe to int64 for merging with page_data
prediction['rev_id'] = prediction['rev_id'].astype('int64')

# Merge the perdiction back to the page_data
pred_page = pd.merge(left=page_data, right=prediction, left_on='rev_id', right_on='rev_id')

In [80]:
# Check if all rows are assigned a prediction
pred_page[pred_page['prediction'].isna()]

,page,country,rev_id,prediction


In [81]:
pred_page

,page,country,rev_id,prediction
0,Bir I of Kanem,Chad,355319463,Stub
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub
2,Yos Por,Cambodia,393822005,Stub
3,Julius Gregr,Czech Republic,395521877,Stub
4,Edvard Gregr,Czech Republic,395526568,Stub
...,...,...,...,...
46420,Hal Bidlack,United States,807481636,C
46421,Yahya Jammeh,Gambia,807482007,GA
46422,Lucius Fairchild,United States,807483006,C
46423,Fahd of Saudi Arabia,Saudi Arabia,807483153,GA


In [82]:
# Combine the WPDS data with the page data with prediction

full_res = pd.merge(left=pred_page, right=WPDS_clean, how='outer', left_on='country', right_on='Name')

In [83]:
full_res

,page,country,rev_id,prediction,FIPS,Name,Type,TimeFrame,Data (M),Population
0,Bir I of Kanem,Chad,355319463.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0
1,Abdullah II of Kanem,Chad,498683267.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0
2,Salmama II of Kanem,Chad,565745353.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0
3,Kuri I of Kanem,Chad,565745365.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0
4,Mohammed I of Kanem,Chad,565745375.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0
...,...,...,...,...,...,...,...,...,...,...
46446,NaN,NaN,NaN,NaN,PF,French Polynesia,Country,2019.0,0.280,280000.0
46447,NaN,NaN,NaN,NaN,GU,Guam,Country,2019.0,0.175,175000.0
46448,NaN,NaN,NaN,NaN,NC,New Caledonia,Country,2019.0,0.295,295000.0
46449,NaN,NaN,NaN,NaN,PW,Palau,Country,2019.0,0.018,18000.0


In [84]:
# Extract the rows with a null country or Name (no matching between WPDS and page data)
no_match = full_res[full_res['country'].isna()|full_res['Name'].isna()].reset_index()

In [85]:
# if wanted, the following code can be used to create a new columns in replace of Name and country (since there are no overlapping)
# no_match['new_country']=[no_match['Name'][i] if pd.isna(no_match['country'][i]) else no_match['country'][i] for i in range(len(no_match))]

In [86]:
# Output the csv file for no matching record 
no_match.to_csv('wp_wpds_countries-no_match.csv')

In [87]:
# Extract rows with matching record between WPDS and page data
match_record = pd.merge(left=pred_page, right=WPDS_clean, how='inner', left_on='country', right_on='Name')

# Select and rename interested columns 
match_record_out = match_record[['country','page','rev_id','prediction','Population']]

match_record_out.columns = ['country','article_name','revision_id','article_quality_est.','population']

match_record_out

,country,article_name,revision_id,article_quality_est.,population
0,Chad,Bir I of Kanem,355319463,Stub,16877000
1,Chad,Abdullah II of Kanem,498683267,Stub,16877000
2,Chad,Salmama II of Kanem,565745353,Stub,16877000
3,Chad,Kuri I of Kanem,565745365,Stub,16877000
4,Chad,Mohammed I of Kanem,565745375,Stub,16877000
...,...,...,...,...,...
44563,Seychelles,Rita Sinon,800323154,Stub,98000
44564,Seychelles,Sylvette Frichot,800323798,Stub,98000
44565,Seychelles,May De Silva,800969960,Start,98000
44566,Seychelles,Vincent Meriton,802051093,Stub,98000


In [88]:
# Output the matching record to csv file

match_record_out.to_csv('wp_wpds_politicians_by_country.csv')

## Analysis

In [89]:
# Count the number of articles for each country 
count = pd.DataFrame(match_record_out.groupby('country').size()).reset_index()
count.columns=['country','article_count']

count

,country,article_count
0,Afghanistan,319
1,Albania,456
2,Algeria,116
3,Andorra,34
4,Angola,106
...,...,...
178,Venezuela,130
179,Vietnam,187
180,Yemen,116
181,Zambia,25


In [90]:
# Count the number of high quality articles (FA or GA class) for each country 

count_high_quality = pd.DataFrame(match_record_out[match_record_out['article_quality_est.'].isin(['FA','GA'])].groupby('country').size()).reset_index()

count_high_quality.columns=['country','high_quality_article_count']

count_high_quality

,country,high_quality_article_count
0,Afghanistan,13
1,Albania,3
2,Algeria,2
3,Argentina,16
4,Armenia,5
...,...,...
141,Vanuatu,3
142,Venezuela,3
143,Vietnam,13
144,Yemen,3


In [91]:
# Check the layout of the subregion and the hierarchical structure

WPDS_data[WPDS_data['Type']=='Sub-Region']

,FIPS,Name,Type,TimeFrame,Data (M),Population,region,sub-region
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,AFRICA,NaN
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,AFRICA,NORTHERN AFRICA
10,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000,AFRICA,WESTERN AFRICA
27,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000,AFRICA,EASTERN AFRICA
48,MIDDLE AFRICA,MIDDLE AFRICA,Sub-Region,2019,179.757,179757000,AFRICA,MIDDLE AFRICA
58,SOUTHERN AFRICA,SOUTHERN AFRICA,Sub-Region,2019,67.732,67732000,AFRICA,SOUTHERN AFRICA
64,NORTHERN AMERICA,NORTHERN AMERICA,Sub-Region,2019,368.193,368193000,NORTHERN AMERICA,SOUTHERN AFRICA
67,LATIN AMERICA AND THE CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Sub-Region,2019,651.036,651036000,LATIN AMERICA AND THE CARIBBEAN,SOUTHERN AFRICA
68,CENTRAL AMERICA,CENTRAL AMERICA,Sub-Region,2019,178.611,178611000,LATIN AMERICA AND THE CARIBBEAN,CENTRAL AMERICA
77,CARIBBEAN,CARIBBEAN,Sub-Region,2019,43.233,43233000,LATIN AMERICA AND THE CARIBBEAN,CARIBBEAN


In [92]:
# Combine the count tables to the population data 

count_combined = pd.merge(left=count, right=count_high_quality, left_on='country', right_on='country')

In [93]:
# Extract the population data for each country and he sub-region it belongs to 

region = WPDS_data[WPDS_data['Name'].isin(['AFRICA','NORTHERN AMERICA','LATIN AMERICA AND THE CARIBBEAN','SOUTH AMERICA','ASIA','EUROPE','OCEANIA'])]
region

,FIPS,Name,Type,TimeFrame,Data (M),Population,region,sub-region
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,AFRICA,NaN
64,NORTHERN AMERICA,NORTHERN AMERICA,Sub-Region,2019,368.193,368193000,NORTHERN AMERICA,SOUTHERN AFRICA
67,LATIN AMERICA AND THE CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Sub-Region,2019,651.036,651036000,LATIN AMERICA AND THE CARIBBEAN,SOUTHERN AFRICA
95,SOUTH AMERICA,SOUTH AMERICA,Sub-Region,2019,429.191,429191000,SOUTH AMERICA,CARIBBEAN
109,ASIA,ASIA,Sub-Region,2019,4625.927,4625927000,ASIA,CARIBBEAN
166,EUROPE,EUROPE,Sub-Region,2019,746.622,746622000,EUROPE,EAST ASIA
216,OCEANIA,OCEANIA,Sub-Region,2019,43.155,43155000,OCEANIA,NaN


In [94]:
# Create an empty column to store the region information 
WPDS_data['region'] = np.nan

In [95]:
# Assign the region for each row based on the row index 

index_list = region.index.values.tolist()
for i in range(len(index_list)):
    if i < len(index_list)-1:
        WPDS_data['region'][index_list[i]:index_list[i+1]] = region['Name'][index_list[i]]
    else:
        WPDS_data['region'][index_list[i]:] = region['Name'][index_list[i]] 

<ipython-input-95-e36cabec9999>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WPDS_data['region'][index_list[i]:index_list[i+1]] = region['Name'][index_list[i]]
<ipython-input-95-e36cabec9999>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WPDS_data['region'][index_list[i]:] = region['Name'][index_list[i]]


In [96]:
# Find the indexes for all sub-regions 

temp = WPDS_data[WPDS_data['Type']=='Sub-Region']
subregion = temp[~temp['Name'].isin(['AFRICA','NORTHERN AMERICA','LATIN AMERICA AND THE CARIBBEAN','SOUTH AMERICA','ASIA','EUROPE','OCEANIA','Channel Islands'])]
subregion

,FIPS,Name,Type,TimeFrame,Data (M),Population,region,sub-region
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,AFRICA,NORTHERN AFRICA
10,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000,AFRICA,WESTERN AFRICA
27,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000,AFRICA,EASTERN AFRICA
48,MIDDLE AFRICA,MIDDLE AFRICA,Sub-Region,2019,179.757,179757000,AFRICA,MIDDLE AFRICA
58,SOUTHERN AFRICA,SOUTHERN AFRICA,Sub-Region,2019,67.732,67732000,AFRICA,SOUTHERN AFRICA
68,CENTRAL AMERICA,CENTRAL AMERICA,Sub-Region,2019,178.611,178611000,LATIN AMERICA AND THE CARIBBEAN,CENTRAL AMERICA
77,CARIBBEAN,CARIBBEAN,Sub-Region,2019,43.233,43233000,LATIN AMERICA AND THE CARIBBEAN,CARIBBEAN
110,WESTERN ASIA,WESTERN ASIA,Sub-Region,2019,280.927,280927000,ASIA,WESTERN ASIA
129,CENTRAL ASIA,CENTRAL ASIA,Sub-Region,2019,74.961,74961000,ASIA,CENTRAL ASIA
135,SOUTH ASIA,SOUTH ASIA,Sub-Region,2019,1967.131,1967131000,ASIA,SOUTH ASIA


In [97]:
# Create an empty column to store the sub-region information 
WPDS_data['sub-region'] = np.nan

In [98]:
# Assign the sub-region for each row based on the row index 

index_list = subregion.index.values.tolist()
for i in range(len(index_list)):
    if i < len(index_list)-1:
        WPDS_data['sub-region'][index_list[i]:index_list[i+1]] = subregion['Name'][index_list[i]]
    else: 
        WPDS_data['sub-region'][index_list[i]:216] = subregion['Name'][index_list[i]]

<ipython-input-98-48b6b2eea333>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WPDS_data['sub-region'][index_list[i]:index_list[i+1]] = subregion['Name'][index_list[i]]
<ipython-input-98-48b6b2eea333>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WPDS_data['sub-region'][index_list[i]:216] = subregion['Name'][index_list[i]]


In [99]:
WPDS_data

,FIPS,Name,Type,TimeFrame,Data (M),Population,region,sub-region
0,WORLD,WORLD,World,2019,7772.850,7772850000,NaN,NaN
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,AFRICA,NaN
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,AFRICA,NORTHERN AFRICA
3,DZ,Algeria,Country,2019,44.357,44357000,AFRICA,NORTHERN AFRICA
4,EG,Egypt,Country,2019,100.803,100803000,AFRICA,NORTHERN AFRICA
...,...,...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000,OCEANIA,NaN
230,SB,Solomon Islands,Country,2019,0.715,715000,OCEANIA,NaN
231,TO,Tonga,Country,2019,0.099,99000,OCEANIA,NaN
232,TV,Tuvalu,Country,2019,0.010,10000,OCEANIA,NaN


In [100]:
# Merge all three dataset together to obtain the final table needed for analysis 

final_count = pd.merge(left=WPDS_data[['Name','Population','sub-region','region']], right=count_combined, how='right', left_on='Name', right_on='country')

In [101]:
final_count

,Name,Population,sub-region,region,country,article_count,high_quality_article_count
0,Afghanistan,38928000,SOUTH ASIA,ASIA,Afghanistan,319,13
1,Albania,2838000,SOUTHERN EUROPE,EUROPE,Albania,456,3
2,Algeria,44357000,NORTHERN AFRICA,AFRICA,Algeria,116,2
3,Argentina,45377000,CARIBBEAN,SOUTH AMERICA,Argentina,491,16
4,Armenia,2956000,WESTERN ASIA,ASIA,Armenia,193,5
...,...,...,...,...,...,...,...
141,Vanuatu,321000,NaN,OCEANIA,Vanuatu,58,3
142,Venezuela,28645000,CARIBBEAN,SOUTH AMERICA,Venezuela,130,3
143,Vietnam,96209000,SOUTHEAST ASIA,ASIA,Vietnam,187,13
144,Yemen,29826000,WESTERN ASIA,ASIA,Yemen,116,3


In [102]:
# Calculate the percentage per country 

country_proportion = final_count.copy()
country_proportion['article_per_population']= country_proportion['article_count']/country_proportion['Population']
country_proportion['high_quality_article_per_article']= country_proportion['high_quality_article_count']/country_proportion['article_count']

In [103]:
country_proportion.sort_values(by=['article_per_population'], ascending=False)

,Name,Population,sub-region,region,country,article_count,high_quality_article_count,article_per_population,high_quality_article_per_article
133,Tuvalu,10000,NaN,OCEANIA,Tuvalu,54,4,5.400000e-03,0.074074
51,Iceland,368000,NORTHERN EUROPE,EUROPE,Iceland,201,2,5.461957e-04,0.009950
74,Luxembourg,632000,WESTERN EUROPE,EUROPE,Luxembourg,178,1,2.816456e-04,0.005618
39,Fiji,896000,NaN,OCEANIA,Fiji,197,1,2.198661e-04,0.005076
80,Malta,522000,SOUTHERN EUROPE,EUROPE,Malta,101,1,1.934866e-04,0.009901
...,...,...,...,...,...,...,...,...,...
38,Ethiopia,114916000,EASTERN AFRICA,AFRICA,Ethiopia,101,2,8.789029e-07,0.019802
140,Uzbekistan,34174000,CENTRAL ASIA,ASIA,Uzbekistan,28,3,8.193363e-07,0.107143
26,China,1402385000,EAST ASIA,ASIA,China,1129,40,8.050571e-07,0.035430
53,Indonesia,271739000,SOUTHEAST ASIA,ASIA,Indonesia,209,9,7.691204e-07,0.043062


In [104]:
# Calculate the percentage per region (using total population from the original WPDS data)

region_proportion = pd.merge(pd.merge(pd.DataFrame(final_count.groupby('region')['article_count'].sum()).reset_index(),
         pd.DataFrame(final_count.groupby('region')['high_quality_article_count'].sum()).reset_index(),
         left_on='region', right_on='region'),
         region[['Name','Population']], left_on='region', right_on='Name').drop(['Name'], axis=1)

region_proportion['article_per_population']= region_proportion['article_count']/region_proportion['Population']
region_proportion['high_quality_article_per_article']= region_proportion['high_quality_article_count']/region_proportion['article_count']

In [105]:
region_proportion.sort_values(by=['article_per_population'], ascending=False)

,region,article_count,high_quality_article_count,Population,article_per_population,high_quality_article_per_article
5,OCEANIA,2811,63,43155000,0.000065,0.022412
2,EUROPE,14444,350,746622000,0.000019,0.024232
6,SOUTH AMERICA,2985,40,429191000,0.000007,0.013400
4,NORTHERN AMERICA,1901,104,368193000,0.000005,0.054708
0,AFRICA,6139,119,1337918000,0.000005,0.019384
3,LATIN AMERICA AND THE CARIBBEAN,1932,36,651036000,0.000003,0.018634
1,ASIA,11515,316,4625927000,0.000002,0.027442


In [106]:
# Calculate the percentage per sub-region (using total population from the original WPDS data)

subregion_proportion = pd.merge(pd.merge(pd.DataFrame(final_count.groupby('sub-region')['article_count'].sum()).reset_index(),
         pd.DataFrame(final_count.groupby('sub-region')['high_quality_article_count'].sum()).reset_index(),
         left_on='sub-region', right_on='sub-region'),
         subregion[['Name','Population']], left_on='sub-region', right_on='Name').drop(['Name'], axis=1)

subregion_proportion['article_per_population']= subregion_proportion['article_count']/subregion_proportion['Population']
subregion_proportion['high_quality_article_per_article']= subregion_proportion['high_quality_article_count']/subregion_proportion['article_count']

In [107]:
# combine region_proportion and subregion_proportion after renaming the first column to 'region/subregion'

region1 = region_proportion.copy()
region1.rename(columns={'region': 'region/subregion'}, inplace=True)
region2 = subregion_proportion.copy()
region2.rename(columns={'sub-region': 'region/subregion'}, inplace=True)


full_region = pd.concat([region1,region2])

## Results

### Top 10 countries by coverage

In [108]:
country_proportion.sort_values(by=['article_per_population'], ascending=False).head(10)

,Name,Population,sub-region,region,country,article_count,high_quality_article_count,article_per_population,high_quality_article_per_article
133,Tuvalu,10000,NaN,OCEANIA,Tuvalu,54,4,0.005400,0.074074
51,Iceland,368000,NORTHERN EUROPE,EUROPE,Iceland,201,2,0.000546,0.009950
74,Luxembourg,632000,WESTERN EUROPE,EUROPE,Luxembourg,178,1,0.000282,0.005618
39,Fiji,896000,NaN,OCEANIA,Fiji,197,1,0.000220,0.005076
80,Malta,522000,SOUTHERN EUROPE,EUROPE,Malta,101,1,0.000193,0.009901
141,Vanuatu,321000,NaN,OCEANIA,Vanuatu,58,3,0.000181,0.051724
33,Dominica,72000,CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Dominica,12,1,0.000167,0.083333
1,Albania,2838000,SOUTHERN EUROPE,EUROPE,Albania,456,3,0.000161,0.006579
91,New Zealand,4987000,NaN,OCEANIA,New Zealand,783,13,0.000157,0.016603
78,Maldives,541000,SOUTH ASIA,ASIA,Maldives,83,1,0.000153,0.012048


### Bottom 10 countries by coverage

In [109]:
country_proportion.sort_values(by=['article_per_population'], ascending=True).head(10)

,Name,Population,sub-region,region,country,article_count,high_quality_article_count,article_per_population,high_quality_article_per_article
52,India,1400100000,SOUTH ASIA,ASIA,India,968,13,6.913792e-07,0.013430
53,Indonesia,271739000,SOUTHEAST ASIA,ASIA,Indonesia,209,9,7.691204e-07,0.043062
26,China,1402385000,EAST ASIA,ASIA,China,1129,40,8.050571e-07,0.035430
140,Uzbekistan,34174000,CENTRAL ASIA,ASIA,Uzbekistan,28,3,8.193363e-07,0.107143
38,Ethiopia,114916000,EASTERN AFRICA,AFRICA,Ethiopia,101,2,8.789029e-07,0.019802
63,"Korea, North",25779000,EAST ASIA,ASIA,"Korea, North",36,8,1.396486e-06,0.222222
129,Thailand,66534000,SOUTHEAST ASIA,ASIA,Thailand,112,3,1.683350e-06,0.026786
8,Bangladesh,169809000,SOUTH ASIA,ASIA,Bangladesh,317,3,1.866803e-06,0.009464
143,Vietnam,96209000,SOUTHEAST ASIA,ASIA,Vietnam,187,13,1.943685e-06,0.069519
121,Sudan,43849000,NORTHERN AFRICA,AFRICA,Sudan,95,2,2.166526e-06,0.021053


### Top 10 countries by relative quality

In [110]:
country_proportion.sort_values(by=['high_quality_article_per_article'], ascending=False).head(10)

,Name,Population,sub-region,region,country,article_count,high_quality_article_count,article_per_population,high_quality_article_per_article
63,"Korea, North",25779000,EAST ASIA,ASIA,"Korea, North",36,8,1.396486e-06,0.222222
109,Saudi Arabia,35041000,WESTERN ASIA,ASIA,Saudi Arabia,117,15,3.338946e-06,0.128205
106,Romania,19241000,EASTERN EUROPE,EUROPE,Romania,343,42,1.782652e-05,0.122449
23,Central African Republic,4830000,MIDDLE AFRICA,AFRICA,Central African Republic,66,8,1.366460e-05,0.121212
140,Uzbekistan,34174000,CENTRAL ASIA,ASIA,Uzbekistan,28,3,8.193363e-07,0.107143
82,Mauritania,4650000,WESTERN AFRICA,AFRICA,Mauritania,48,5,1.032258e-05,0.104167
46,Guatemala,18066000,CENTRAL AMERICA,LATIN AMERICA AND THE CARIBBEAN,Guatemala,83,7,4.594265e-06,0.084337
33,Dominica,72000,CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Dominica,12,1,1.666667e-04,0.083333
125,Syria,19398000,WESTERN ASIA,ASIA,Syria,128,10,6.598618e-06,0.078125
11,Benin,12209000,WESTERN AFRICA,AFRICA,Benin,91,7,7.453518e-06,0.076923


### Bottom 10 countries by relative quality

In [111]:
country_proportion.sort_values(by=['high_quality_article_per_article'], ascending=True).head(10)

,Name,Population,sub-region,region,country,article_count,high_quality_article_count,article_per_population,high_quality_article_per_article
10,Belgium,11515000,WESTERN EUROPE,EUROPE,Belgium,519,1,0.000045,0.001927
128,Tanzania,59734000,EASTERN AFRICA,AFRICA,Tanzania,404,1,0.000007,0.002475
124,Switzerland,8634000,WESTERN EUROPE,EUROPE,Switzerland,402,1,0.000047,0.002488
89,Nepal,29996000,SOUTH ASIA,ASIA,Nepal,356,1,0.000012,0.002809
101,Peru,32824000,CARIBBEAN,SOUTH AMERICA,Peru,350,1,0.000011,0.002857
94,Nigeria,206140000,WESTERN AFRICA,AFRICA,Nigeria,676,2,0.000003,0.002959
104,Portugal,10255000,SOUTHERN EUROPE,EUROPE,Portugal,318,1,0.000031,0.003145
27,Colombia,49444000,CARIBBEAN,SOUTH AMERICA,Colombia,285,1,0.000006,0.003509
73,Lithuania,2794000,NORTHERN EUROPE,EUROPE,Lithuania,244,1,0.000087,0.004098
87,Morocco,35952000,NORTHERN AFRICA,AFRICA,Morocco,206,1,0.000006,0.004854


### Geographic regions by coverage
#### Ranking of geographic regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population


In [112]:
# continent level 
region_proportion.sort_values(by=['article_per_population'], ascending=False)

,region,article_count,high_quality_article_count,Population,article_per_population,high_quality_article_per_article
5,OCEANIA,2811,63,43155000,0.000065,0.022412
2,EUROPE,14444,350,746622000,0.000019,0.024232
6,SOUTH AMERICA,2985,40,429191000,0.000007,0.013400
4,NORTHERN AMERICA,1901,104,368193000,0.000005,0.054708
0,AFRICA,6139,119,1337918000,0.000005,0.019384
3,LATIN AMERICA AND THE CARIBBEAN,1932,36,651036000,0.000003,0.018634
1,ASIA,11515,316,4625927000,0.000002,0.027442


In [113]:
# sub-region level 
subregion_proportion.sort_values(by=['article_per_population'], ascending=False).head(10)

,sub-region,article_count,high_quality_article_count,Population,article_per_population,high_quality_article_per_article
0,CARIBBEAN,3537,53,43233000,0.000082,0.014984
11,SOUTHERN AFRICA,2344,113,67732000,0.000035,0.048208
8,NORTHERN EUROPE,3046,102,105990000,0.000029,0.033487
12,SOUTHERN EUROPE,3595,74,153251000,0.000023,0.020584
15,WESTERN EUROPE,4492,56,195479000,0.000023,0.012467
5,EASTERN EUROPE,3311,118,291902000,0.000011,0.035639
14,WESTERN ASIA,2521,89,280927000,0.000009,0.035303
1,CENTRAL AMERICA,1380,23,178611000,0.000008,0.016667
13,WESTERN AFRICA,2103,40,401115000,0.000005,0.019020
4,EASTERN AFRICA,2294,35,444970000,0.000005,0.015257


In [114]:
# all regions/subregions together

full_region.sort_values(by=['article_per_population'], ascending=False).head(10)

,region/subregion,article_count,high_quality_article_count,Population,article_per_population,high_quality_article_per_article
0,CARIBBEAN,3537,53,43233000,0.000082,0.014984
5,OCEANIA,2811,63,43155000,0.000065,0.022412
11,SOUTHERN AFRICA,2344,113,67732000,0.000035,0.048208
8,NORTHERN EUROPE,3046,102,105990000,0.000029,0.033487
12,SOUTHERN EUROPE,3595,74,153251000,0.000023,0.020584
15,WESTERN EUROPE,4492,56,195479000,0.000023,0.012467
2,EUROPE,14444,350,746622000,0.000019,0.024232
5,EASTERN EUROPE,3311,118,291902000,0.000011,0.035639
14,WESTERN ASIA,2521,89,280927000,0.000009,0.035303
1,CENTRAL AMERICA,1380,23,178611000,0.000008,0.016667


### Geographic regions by coverage II
#### Ranking of geographic regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

In [115]:
# continent level
region_proportion.sort_values(by=['high_quality_article_per_article'], ascending=False)

,region,article_count,high_quality_article_count,Population,article_per_population,high_quality_article_per_article
4,NORTHERN AMERICA,1901,104,368193000,0.000005,0.054708
1,ASIA,11515,316,4625927000,0.000002,0.027442
2,EUROPE,14444,350,746622000,0.000019,0.024232
5,OCEANIA,2811,63,43155000,0.000065,0.022412
0,AFRICA,6139,119,1337918000,0.000005,0.019384
3,LATIN AMERICA AND THE CARIBBEAN,1932,36,651036000,0.000003,0.018634
6,SOUTH AMERICA,2985,40,429191000,0.000007,0.013400


In [116]:
# sub-region level 
subregion_proportion.sort_values(by=['high_quality_article_per_article'], ascending=False).head(10)

,sub-region,article_count,high_quality_article_count,Population,article_per_population,high_quality_article_per_article
2,CENTRAL ASIA,135,7,74961000,0.000002,0.051852
11,SOUTHERN AFRICA,2344,113,67732000,0.000035,0.048208
10,SOUTHEAST ASIA,2020,73,661845000,0.000003,0.036139
5,EASTERN EUROPE,3311,118,291902000,0.000011,0.035639
14,WESTERN ASIA,2521,89,280927000,0.000009,0.035303
8,NORTHERN EUROPE,3046,102,105990000,0.000029,0.033487
3,EAST ASIA,2473,76,1641063000,0.000002,0.030732
6,MIDDLE AFRICA,538,16,179757000,0.000003,0.029740
7,NORTHERN AFRICA,761,19,244344000,0.000003,0.024967
12,SOUTHERN EUROPE,3595,74,153251000,0.000023,0.020584


In [117]:
# all regions/subregions together 

full_region.sort_values(by=['high_quality_article_per_article'], ascending=False).head(10)

,region/subregion,article_count,high_quality_article_count,Population,article_per_population,high_quality_article_per_article
4,NORTHERN AMERICA,1901,104,368193000,0.000005,0.054708
2,CENTRAL ASIA,135,7,74961000,0.000002,0.051852
11,SOUTHERN AFRICA,2344,113,67732000,0.000035,0.048208
10,SOUTHEAST ASIA,2020,73,661845000,0.000003,0.036139
5,EASTERN EUROPE,3311,118,291902000,0.000011,0.035639
14,WESTERN ASIA,2521,89,280927000,0.000009,0.035303
8,NORTHERN EUROPE,3046,102,105990000,0.000029,0.033487
3,EAST ASIA,2473,76,1641063000,0.000002,0.030732
6,MIDDLE AFRICA,538,16,179757000,0.000003,0.029740
1,ASIA,11515,316,4625927000,0.000002,0.027442


#### Appendix: Checking if there are countries using "Channel Islands" as country name 

In [123]:
page_data[page_data['country']=='Channel Islands']

,page,country,rev_id
